In [2]:
!pip install dagshub

  Using cached dacite-1.6.0-py3-none-any.whl.metadata (14 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached backoff-2.2.1-py3-none-any.whl.metadata (14 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
Using cached dacite-1.6.0-py3-none-any.whl (12 kB)
Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
Using cached backoff-2.2.1-py3-none-any.whl (15 kB)
Using cached markdown_it_py-3.0.0-py3-none-any.whl (87 kB)
Using cached typing_inspect-0.9.0-py3-none-any.whl (8.8 kB)
   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   ---------------------------------------- 3.8/3.8 MB 25.2 MB/s eta 0:00:00
Using cached mdurl-0.1.2-py3-none-any.whl (10.0 kB)
  Attempting uninstall: graphql-core
    Found existing installation: graphql-core 3.2.6
    Uninstalling graphql-core

In [ ]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

# Now you can import from src
from src.data_utils import load_and_process_taxi_data

import pandas as pd

import dagshub
dagshub.init(repo_owner="sahilsubhasbhaivachhani", repo_name="sp25_taxi", mlflow=True)

In [4]:
rides1 = load_and_process_taxi_data(year=2022)
rides2=(load_and_process_taxi_data(year=2023))
rides = pd.concat([rides1,rides2],ignore_index=True)

Successfully downloaded data for 2022-01.
Loading data for 2022-01...
Total records: 2,463,931
Valid records: 2,415,141
Records dropped: 48,790 (1.98%)
Successfully processed data for 2022-01.
Successfully downloaded data for 2022-02.
Loading data for 2022-02...
Total records: 2,979,431
Valid records: 2,921,118
Records dropped: 58,313 (1.96%)
Successfully processed data for 2022-02.
Successfully downloaded data for 2022-03.
Loading data for 2022-03...
Total records: 3,627,882
Valid records: 3,551,986
Records dropped: 75,896 (2.09%)
Successfully processed data for 2022-03.
Successfully downloaded data for 2022-04.
Loading data for 2022-04...
Total records: 3,599,920
Valid records: 3,522,113
Records dropped: 77,807 (2.16%)
Successfully processed data for 2022-04.
Successfully downloaded data for 2022-05.
Loading data for 2022-05...
Total records: 3,588,295
Valid records: 3,509,056
Records dropped: 79,239 (2.21%)
Successfully processed data for 2022-05.
Successfully downloaded data for 20

In [5]:
temp_rides = rides[rides["pickup_location_id"] == 43]

In [6]:
from src.data_utils import transform_raw_data_into_ts_data

ts_data = transform_raw_data_into_ts_data(temp_rides)
ts_data.head()

C:\Users\sahil\Desktop\github\sp25_taxi\src\data_utils.py:223: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rides["pickup_hour"] = rides["pickup_datetime"].dt.floor("h")


,pickup_hour,pickup_location_id,rides
0,2022-01-01 00:00:00,43,96
1,2022-01-01 01:00:00,43,60
2,2022-01-01 02:00:00,43,22
3,2022-01-01 03:00:00,43,8
4,2022-01-01 04:00:00,43,4


In [7]:
ts_data

,pickup_hour,pickup_location_id,rides
0,2022-01-01 00:00:00,43,96
1,2022-01-01 01:00:00,43,60
2,2022-01-01 02:00:00,43,22
3,2022-01-01 03:00:00,43,8
4,2022-01-01 04:00:00,43,4
...,...,...,...
17515,2023-12-31 19:00:00,43,55
17516,2023-12-31 20:00:00,43,72
17517,2023-12-31 21:00:00,43,50
17518,2023-12-31 22:00:00,43,28


In [8]:
ts_data = ts_data.drop(columns=["pickup_location_id"])

In [9]:
print(ts_data.head())


          pickup_hour  rides
0 2022-01-01 00:00:00     96
1 2022-01-01 01:00:00     60
2 2022-01-01 02:00:00     22
3 2022-01-01 03:00:00      8
4 2022-01-01 04:00:00      4


In [11]:
!pip install pmdarima

   ---------------------------------------- 0.0/614.7 kB ? eta -:--:--
   ---------------------------------------- 614.7/614.7 kB 4.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   ---------------------------------------- 2.8/2.8 MB 20.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/9.9 MB ? eta -:--:--
   ---------------------------------------  9.7/9.9 MB 54.9 MB/s eta 0:00:01
   ---------------------------------------- 9.9/9.9 MB 40.8 MB/s eta 0:00:00


In [12]:
from pmdarima import auto_arima

# Find best ARIMA(p, d, q)
model_auto = auto_arima(ts_data["rides"], seasonal=False, stepwise=True, trace=True)

# Print the best parameters
print(model_auto.summary())


C:\Users\sahil\anaconda3\envs\sp25_taxi\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\sahil\anaconda3\envs\sp25_taxi\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\sahil\anaconda3\envs\sp25_taxi\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\sahil\anaconda3\envs\sp25_taxi\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\sahil\anaconda3\envs\sp25_taxi\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure

Performing stepwise search to minimize aic
 ARIMA(2,0,2)(0,0,0)[0]             : AIC=162977.313, Time=2.60 sec
 ARIMA(0,0,0)(0,0,0)[0]             : AIC=210122.120, Time=0.13 sec


C:\Users\sahil\anaconda3\envs\sp25_taxi\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\sahil\anaconda3\envs\sp25_taxi\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


 ARIMA(1,0,0)(0,0,0)[0]             : AIC=165205.279, Time=0.15 sec


C:\Users\sahil\anaconda3\envs\sp25_taxi\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


...

 ARIMA(3,0,5)(0,0,0)[0]             : AIC=159024.825, Time=13.91 sec


C:\Users\sahil\anaconda3\envs\sp25_taxi\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


 ARIMA(5,0,5)(0,0,0)[0]             : AIC=161665.229, Time=14.19 sec


C:\Users\sahil\anaconda3\envs\sp25_taxi\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


 ARIMA(4,0,4)(0,0,0)[0] intercept   : AIC=158527.799, Time=24.00 sec

Best model:  ARIMA(4,0,4)(0,0,0)[0]          
Total fit time: 172.324 seconds
                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                17520
Model:               SARIMAX(4, 0, 4)   Log Likelihood              -78354.220
Date:                Wed, 05 Mar 2025   AIC                         156726.439
Time:                        20:31:39   BIC                         156796.379
Sample:                             0   HQIC                        156749.469
                              - 17520                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          3.2394      0.025    130.476      0.000       3

In [13]:
from statsmodels.tsa.arima.model import ARIMA

# Fit the best ARIMA model (Example: ARIMA(4, 0, 4) if suggested)
model_arima = ARIMA(ts_data["rides"], order=(4, 0, 4))  # Replace with best (p, d, q)
arima_result = model_arima.fit()

# Print model summary
print(arima_result.summary())

C:\Users\sahil\anaconda3\envs\sp25_taxi\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                               SARIMAX Results                                
Dep. Variable:                  rides   No. Observations:                17520
Model:                 ARIMA(4, 0, 4)   Log Likelihood              -78002.497
Date:                Wed, 05 Mar 2025   AIC                         156024.994
Time:                        20:32:25   BIC                         156102.705
Sample:                             0   HQIC                        156050.583
                              - 17520                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         73.4148      0.769     95.443      0.000      71.907      74.922
ar.L1          1.6882      0.007    251.314      0.000       1.675       1.701
ar.L2          0.2264      0.006     35.423      0.0

In [14]:
from sklearn.metrics import mean_absolute_error

# Split data into train (80%) and test (20%)
train_size = int(len(ts_data) * 0.8)
train, test = ts_data.iloc[:train_size], ts_data.iloc[train_size:]

# Fit ARIMA on training data
model_arima = ARIMA(train["rides"], order=(4, 0, 4))  # Use optimal (p,d,q)
arima_result = model_arima.fit()

# Forecast for test set
forecast = arima_result.forecast(steps=len(test))

# Compute MAE
mae = mean_absolute_error(test["rides"], forecast)
print("Mean Absolute Error (MAE) for ARIMA:", mae)


C:\Users\sahil\anaconda3\envs\sp25_taxi\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Mean Absolute Error (MAE) for ARIMA: 26.876948817497336


In [15]:
import mlflow
mlflow.set_experiment("model_arima")

# Start an MLflow run
with mlflow.start_run():
        mlflow.log_metric("mean_absolute_error", mae)

2025/03/05 20:33:15 INFO mlflow.tracking.fluent: Experiment with name 'model_arima' does not exist. Creating a new experiment.


🏃 View run bold-hen-437 at: https://dagshub.com/sahilsubhasbhaivachhani/sp25_taxi.mlflow/#/experiments/6/runs/7f701797716d4f20a6f083ce9366048d
🧪 View experiment at: https://dagshub.com/sahilsubhasbhaivachhani/sp25_taxi.mlflow/#/experiments/6
